In [33]:
import os
import time
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, TensorBoard
from keras.optimizers import Adam
from glob import glob
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split, GridSearchCV, KFold

pd.set_option('max_colwidth', 150)

In [48]:
threshold = 40
path_cnn = '/home/fabiana/Desktop/projeto-final-src/Classifier/InceptionV3_Features/10'
proc_csv = pd.read_csv('../Annotations/processedAnnotations.csv')
count = proc_csv.groupby('CM').count()['palavra']

### Drop duplicated videos

In [41]:
# proc_csv.drop_duplicates('video_url', keep='first', inplace=True)

In [42]:
# proc_csv.to_csv('../Vis/processedAnnotations_no_duplicated_or_corrupted_videos.csv', index=False)

In [36]:
# no_video_list = ['ASFIXIAR', 'ANSIOSO', 'ESPIRITO SANTO(ES)', 'CABELEIREIRO', 'INTESTINO', 
#                  'IMPOTENCIA SEXUAL', 'Y', 'TAMANCO', 'TRAPEZIO', 'TRANSPIRAR', 
#                  'TREINAR', 'BRASIL', 'DIA-1', 'DESENCAMINHAR', 'VARIOS1', 'ViRUS ZIKA', 'PORTA-MALAS']
# proc_csv.drop(proc_csv.query('palavra in @no_video_list').index, inplace=True)

In [39]:
# proc_csv.to_csv('../Vis/processedAnnotations_no_corrupted_videos.csv', index=False)

In [339]:
def replaceClass(cg, threshold):
    quantity = count[cg]
    if (quantity <= threshold):
        return 'others'
    else: 
        return cg

In [340]:
proc_csv['classe'] = proc_csv['CM'].map(lambda x: replaceClass(x, threshold))

In [341]:
def getWord(word):
    return re.search(r".*\/(.*)\.npy", word).group(1)

def getFiles(path):
    files = list(glob(path + "/*.npy"))
    files.sort(key=getWord)
    return files

def generateXY(df, path):
    X = []
    for word in df['palavra']:
        seq = np.load(path + '/' + word + '.npy')
        if seq.shape[0] < 10:
            print(f'{word} -> {seq.shape}')
        X.append(seq)
    return X, df['classe'].astype('category')

def lstm(input_shape, n_classes):
    """Build a simple LSTM network. We pass the extracted features from
    our CNN to this model predomenently."""
    # Model.
    model = Sequential()
    model.add(LSTM(2048, return_sequences=False,
                    input_shape=input_shape,
                    dropout=0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_classes, activation='softmax'))

    return model

In [342]:
X, y_str = generateXY(proc_csv, path_cnn)
X = np.concatenate(X).reshape(-1, X[0].shape[0], X[1].shape[1])
y = to_categorical(y_str.cat.codes)

ABRIR2 -> (9, 2048)
ABSURDO -> (9, 2048)
ACARICIAR -> (9, 2048)
ACATAR -> (9, 2048)
ACENTO2 -> (9, 2048)
ACORDAR1 -> (9, 2048)
ADICIONAR1 -> (9, 2048)
ADMISSAO -> (9, 2048)
ADOLESCENTE1 -> (9, 2048)
ADOTIVO -> (9, 2048)
ADVERTIR2 -> (9, 2048)
AERONAUTICA -> (9, 2048)
AFIADO -> (9, 2048)
AFIXAR1 -> (9, 2048)
AGUENTAR2 -> (9, 2048)
ALARME -> (9, 2048)
ALERTA -> (9, 2048)
ALICATE1 -> (9, 2048)
ALIVIADO -> (9, 2048)
AMAZONAS -> (9, 2048)
ANALFABETO -> (9, 2048)
ANIMAL -> (9, 2048)
ANIVERSARIO1 -> (9, 2048)
ANJO -> (9, 2048)
ANTECEDER -> (9, 2048)
ANUAL -> (9, 2048)
ANUNCIAR -> (9, 2048)
APALPAR -> (9, 2048)
APITO -> (9, 2048)
APONTAR2 -> (9, 2048)
APRECIAR -> (9, 2048)
APRISIONAR -> (9, 2048)
APROXIMAR1 -> (9, 2048)
AQUECER -> (9, 2048)
AQUI -> (9, 2048)
ARARA -> (9, 2048)
ARDER3 -> (9, 2048)
ARREMATAR -> (9, 2048)
ARRUMAR1 -> (9, 2048)
ASPEREZA -> (9, 2048)
ASSISTIR3 -> (9, 2048)
ASSOAR -> (9, 2048)
ASSUMIR3 -> (9, 2048)
ATRAIR -> (9, 2048)
ATRIZ -> (9, 2048)
AVARENTO -> (9, 2048)
AVENIDA

ValueError: cannot reshape array of size 114655232 into shape (10,2048)

In [ ]:
X.shape

In [ ]:
input_shape = (X.shape[1], X.shape[2])
n_classes = y.shape[1]

In [306]:
# cv = KFold(n_splits=5, shuffle=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [316]:
model = 'lstm'

In [317]:
clf = lstm(input_shape, n_classes)

In [318]:
clf.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 2048)              33562624  
_________________________________________________________________
dense_11 (Dense)             (None, 64)                131136    
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 32)                2080      
Total params: 33,695,840
Trainable params: 33,695,840
Non-trainable params: 0
_________________________________________________________________


In [319]:
batch_size = 128
n_epoch = 100

In [320]:
# Helper: Save the model.
checkpointer = ModelCheckpoint(
    filepath=os.path.join('data', 'checkpoints', model + \
        '.{epoch:03d}-{val_loss:.3f}.hdf5'),
    verbose=1,
    save_best_only=True)

# Helper: TensorBoard
# tb = TensorBoard(log_dir=os.path.join('data', 'logs', model))

# Helper: Stop when we stop learning.
early_stopper = EarlyStopping(patience=5)

# Helper: Save results.
timestamp = time.time()
csv_logger = CSVLogger(os.path.join('data', 'logs', model + '-' + 'training-' + \
    str(timestamp) + '.log'))

In [322]:
optimizer = Adam(lr=1e-5, decay=1e-6)
clf.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [323]:
clf.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    validation_data=(X_test, y_test),
    verbose=1,
#     callbacks=[early_stopper, csv_logger, checkpointer],
    epochs=n_epoch)

Train on 3784 samples, validate on 1864 samples
Epoch 1/100
3784/3784 [==============================] - 45s 12ms/step - loss: 3.4261 - accuracy: 0.0756 - val_loss: 3.3120 - val_accuracy: 0.1524
Epoch 2/100
3784/3784 [==============================] - 41s 11ms/step - loss: 3.3730 - accuracy: 0.1089 - val_loss: 3.2916 - val_accuracy: 0.1524
Epoch 3/100
3784/3784 [==============================] - 42s 11ms/step - loss: 3.3680 - accuracy: 0.1068 - val_loss: 3.2745 - val_accuracy: 0.1524
Epoch 4/100
3784/3784 [==============================] - 44s 12ms/step - loss: 3.3580 - accuracy: 0.1065 - val_loss: 3.2562 - val_accuracy: 0.1524
Epoch 5/100
1408/3784 [==========>...................] - ETA: 23s - loss: 3.3559 - accuracy: 0.1165

KeyboardInterrupt: 

In [ ]:
# Use standard fit.
# rm.model.fit(
#     X,
#     y,
#     batch_size=batch_size,
#     validation_data=(X_test, y_test),
#     verbose=1,
#     callbacks=[tb, early_stopper, csv_logger],
#     epochs=nb_epoch)